# FFIEC Data Connect: Example Usage

This notebook demonstrates how to use the `ffiec_data_connect` library to interact with the FFIEC Webservices API. We will cover:

1.  Setting up credentials and a connection.
2.  Retrieving available reporting periods.
3.  Collecting data for a specific institution and reporting period.
4.  Finding filers since a particular date.
5.  Getting filer submission date and time.
6.  Listing all filers for a reporting period.

**Important:** You will need valid credentials (username and password) from the FFIEC to run these examples against the live API. This notebook uses placeholders for these credentials.

In [1]:
import sys
import os

# Assuming your notebook 'example.ipynb' is in the root of the 'ffiec-data-connect' project
# and your source code is in 'ffiec-data-connect/src/'
project_root = os.path.abspath(os.path.join(os.getcwd()))
src_path = os.path.join(project_root, 'src')

# Add the 'src' directory to sys.path if it's not already there
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# You can verify sys.path if needed
# print(sys.path)

# Now your imports like:
# from ffiec_data_connect import methods, credentials, ffiec_connection, exceptions
# should pick up the local versions.

## 1. Imports and Setup

In [2]:
# Standard library imports
from datetime import datetime

# Third-party imports
import requests
import pandas as pd
import polars as pl

# ffiec_data_connect imports
from ffiec_data_connect import methods, credentials, ffiec_connection, exceptions

### 1.1. Credentials

To access the FFIEC Webservices, you need to provide your username and password. The `WebserviceCredentials` class is used for this.

Refer to https://github.com/call-report/ffiec-data-connect for more information on the FFIEC Webservices API.

In [3]:
# Replace with your actual FFIEC webservice username and web token
USER_NAME = None # <- replace with your username
PASSWORD = None # <- replace with your web token (not your web portal password)

# check that a username and password are provided
if USER_NAME is None or PASSWORD is None:
    raise ValueError("Username and password must be provided.")

try:
    creds = credentials.WebserviceCredentials(username=USER_NAME, password=PASSWORD)
    print("Credentials object created successfully.", creds)
except ValueError as e:
    print(f"Error creating credentials: {e}")
    print("Please ensure you provide non-empty username and password.")

Credentials object created successfully. Credentials set from class initialization with username: mlee222


### 1.2. Connection Session

You can use either a standard `requests.Session` or the `FFIECConnection` object from the library. `FFIECConnection` provides additional support for proxies if needed.

All data collection methods in the `methods` module require a session object (`requests.Session` or `ffiec_connection.FFIECConnection`) and a `WebserviceCredentials` object.

In [4]:
# Option 1: Using FFIECConnection (recommended if you might need proxies)
try:
    conn = ffiec_connection.FFIECConnection()
    # You can configure proxy settings on the conn object if needed:
    # conn.use_proxy = True
    # conn.proxy_host = "your_proxy_host"
    # conn.proxy_port = 8080 
    # conn.proxy_protocol = ffiec_connection.ProxyProtocolType.HTTP # or .HTTPS
    # conn.proxy_user_name = "proxy_user" # optional
    # conn.proxy_password = "proxy_pass" # optional
    
    # The actual session object is accessed via conn.session
    ffiec_session = conn.session
    print(f"FFIECConnection created. Session active: {ffiec_session is not None}", ffiec_session)
    # Test connectivity (optional - tests general internet access, not FFIEC service directly without auth)
    # print(f"Connection test successful: {conn.test_connection()}")
except Exception as e:
    print(f"Error creating FFIECConnection: {e}")

# Option 2: Using a standard requests.Session
standard_session = requests.Session()
print(f"Standard requests.Session created: {standard_session is not None}", standard_session)

FFIECConnection created. Session active: True <requests.sessions.Session object at 0x111358150>
Standard requests.Session created: True <requests.sessions.Session object at 0x116bd3350>


For diagnostic purposes, print the services and methods available in the FFIEC Webservice. (See output below).

Each of these functions are mapped to a method in the `methods` module.

In [5]:
print(creds.return_all_services_and_methods(ffiec_session))

Service: RetrievalService
  Method: RetrieveFacsimile
    Input: dataSeries: ns0:ReportingDataSeriesName, reportingPeriodEndDate: xsd:string, fiIDType: ns0:FinancialInstitutionIDType, fiID: xsd:int, facsimileFormat: ns0:FacsimileFormat
    Output: RetrieveFacsimileResult: xsd:base64Binary
--------------------------------
  Method: RetrieveFilersSinceDate
    Input: dataSeries: ns0:ReportingDataSeriesName, reportingPeriodEndDate: xsd:string, lastUpdateDateTime: xsd:string
    Output: RetrieveFilersSinceDateResult: ns0:ArrayOfInt
--------------------------------
  Method: RetrieveFilersSubmissionDateTime
    Input: dataSeries: ns0:ReportingDataSeriesName, reportingPeriodEndDate: xsd:string, lastUpdateDateTime: xsd:string
    Output: RetrieveFilersSubmissionDateTimeResult: ns0:ArrayOfRetrieveFilersDateTime
--------------------------------
  Method: RetrievePanelOfReporters
    Input: dataSeries: ns0:ReportingDataSeriesName, reportingPeriodEndDate: xsd:string
    Output: RetrievePanelOfRep

For the following examples, we'll primarily use the `ffiec_session` from `FFIECConnection`. You can substitute `standard_session` if you prefer and don't need proxy configurations.

**Note:** If you get an error that "Object is not callable", try logging in with your credentials again. This is a known issue with the FFIEC Webservice unrelated to this library.


In [6]:
# In your notebook, after creating creds and ffiec_session
try:
    print("Testing credentials...")
    are_creds_valid = creds.test_credentials(session=ffiec_session)
    print(f"Credentials test result: {are_creds_valid}")
except Exception as e:
    print(f"Error during credentials test: {e}")

# Then try your collect_reporting_periods call
if are_creds_valid: # Or based on direct check if test_credentials doesn't return bool as expected
    reporting_periods = methods.collect_reporting_periods(
        session=ffiec_session,
        creds=creds,
        output_type="list",
        date_output_format="string_original"
    )
    print(reporting_periods)

Testing credentials...
Standby...testing your access.
True
Your credentials are valid.
Credentials test result: None


## 2. Retrieve Reporting Periods

The `collect_reporting_periods` function fetches a list of available reporting period end dates for either 'Call' reports or 'UBPR' (Uniform Bank Performance Report) series.

In [7]:
# Ensure creds and ffiec_session are initialized before running this
if 'creds' in locals() and 'ffiec_session' in locals():
    try:
        print("Fetching Call report periods...")
        # Example 1: Get Call report periods as a list (default output)
        call_periods_list = methods.collect_reporting_periods(
            session=ffiec_session, 
            creds=creds, 
            series="call"
        )
        print(f"First 5 Call periods (list): {call_periods_list[:5]}")
        
        print("\nFetching UBPR report periods as Pandas Series with YYYYMMDD format...")
        # Example 2: Get UBPR report periods as a Pandas Series, dates formatted as YYYYMMDD
        ubpr_periods_pandas = methods.collect_reporting_periods(
            session=ffiec_session, 
            creds=creds, 
            series="ubpr", 
            output_type="pandas",
            date_output_format="string_yyyymmdd"
        )
        print(f"First 5 UBPR periods (Pandas Series):\n{ubpr_periods_pandas.head()}")
        
        print("\nFetching Call report periods as Polars Series with Python datetime objects...")
        # Example 3: Get Call report periods as a Polars Series, dates as Python datetime objects
        call_periods_polars_datetime = methods.collect_reporting_periods(
            session=ffiec_session, 
            creds=creds, 
            series="call",
            output_type="polars",
            date_output_format="python_format"
        )
        print(f"First 5 Call periods (Polars Series, datetime):\n{call_periods_polars_datetime.head()}")
        
    except requests.exceptions.ConnectionError as e:
        print(f"A connection error occurred: {e}. Ensure network connectivity and FFIEC service availability.")
    except exceptions.NoDataError as e: # Assuming a specific exception for no data
        print(f"No data returned: {e}")
    except ValueError as e: # For validation errors like bad credentials (though Zeep might raise others)
        print(f"A value error occurred: {e}. This might be due to invalid credentials or parameters.")
    except Exception as e: # Catch-all for other issues, like Zeep exceptions from the SOAP service
        print(f"An unexpected error occurred: {e}. This could be an issue with the FFIEC webservice response or credentials.")
else:
    print("Skipping `collect_reporting_periods` examples as `creds` or `ffiec_session` not initialized.")

Fetching Call report periods...
First 5 Call periods (list): ['3/31/2025', '12/31/2024', '9/30/2024', '6/30/2024', '3/31/2024']

Fetching UBPR report periods as Pandas Series with YYYYMMDD format...
First 5 UBPR periods (Pandas Series):
0    12/31/2002
1     3/31/2003
2     6/30/2003
3     9/30/2003
4    12/31/2003
Name: reporting_period, dtype: object

Fetching Call report periods as Polars Series with Python datetime objects...
First 5 Call periods (Polars Series, datetime):
shape: (10,)
Series: 'reporting_period' [datetime[μs]]
[
	2025-03-31 00:00:00
	2024-12-31 00:00:00
	2024-09-30 00:00:00
	2024-06-30 00:00:00
	2024-03-31 00:00:00
	2023-12-31 00:00:00
	2023-09-30 00:00:00
	2023-06-30 00:00:00
	2023-03-31 00:00:00
	2022-12-31 00:00:00
]


## 3. Collect Data (Facsimile)

The `collect_data` function retrieves a facsimile (an XBRL document containing the reported data) for a specific financial institution (identified by RSSD ID), reporting period, and data series ('Call' or 'UBPR').

The XBRL data is processed and returned as a list of dictionaries, or a Pandas/Polars DataFrame.

In [8]:
if 'creds' in locals() and 'ffiec_session' in locals() and 'call_periods_list' in locals() and call_periods_list:
    # Use a known RSSD ID for a bank that files Call Reports. Replace with a relevant ID.
    # This is a placeholder. For a real test, you'd need a valid RSSD_ID that exists.
    RSSD_ID_EXAMPLE = "37" # Example: Bank of Hancock County (Sparta, GA) - a good can
    
    # Use the most recent reporting period obtained earlier, or a specific one
    # For FFIEC dates, ensure they are actual quarter-end dates.
    # The function can take various date formats: "MM/DD/YYYY", "YYYY-MM-DD", "YYYYMMDD", "#QYYYY", or datetime object.
    REPORTING_PERIOD_EXAMPLE = call_periods_list[1] # Takes the first period from the list obtained above
    # Or, specify one, e.g., "12/31/2023" or "4Q2023"
    
    try:
        print(f"Fetching Call data for RSSD {RSSD_ID_EXAMPLE} for period {REPORTING_PERIOD_EXAMPLE} as Pandas DataFrame...")
        # Example 1: Call data as Pandas DataFrame
        call_data_df = methods.collect_data(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_EXAMPLE, # e.g., "03/31/2023" or "1Q2023"
            rssd_id=RSSD_ID_EXAMPLE, 
            series="call",
            output_type="pandas",
            date_output_format="python_format" # Get dates as datetime objects in the DataFrame
        )
        if not call_data_df.empty:
            print(f"Successfully fetched Call data. Shape: {call_data_df.shape}")
            print(f"First 5 rows:\n{call_data_df.head()}")
        else:
            print("No data returned for the specified parameters.")

        # Example 2: UBPR data as a list of dictionaries (if you have UBPR access and a relevant RSSD)
        # Note: UBPR data might require different RSSD IDs or might not be available for all institutions.
        print(f"\nFetching UBPR data for RSSD {RSSD_ID_EXAMPLE} for period {REPORTING_PERIOD_EXAMPLE} as list...")
        ubpr_data_list = methods.collect_data(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_EXAMPLE, 
            rssd_id=RSSD_ID_EXAMPLE, 
            series="ubpr",
            output_type="list",
            date_output_format="python_format"
        )
        if ubpr_data_list:
            print(f"Successfully fetched UBPR data. Number of items: {len(ubpr_data_list)}")
            print(f"First item:\n{ubpr_data_list[0]}")
        else:
            print("No UBPR data returned for the specified parameters.")
            
            
        # Example 3: Call data as a Polars DataFrame
        print(f"\nFetching Call data for RSSD {RSSD_ID_EXAMPLE} for period {REPORTING_PERIOD_EXAMPLE} as Polars DataFrame...")
        call_data_polars = methods.collect_data(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_EXAMPLE,
            rssd_id=RSSD_ID_EXAMPLE,
            series="call",
            output_type="polars",
    
        )
        
        print(f"Successfully fetched Call data. Shape: {call_data_polars.shape}")
        print(f"First 5 rows:\n{call_data_polars.head()}")            
        print("call_data_polars", call_data_polars)
            
    except requests.exceptions.ConnectionError as e:
        print(f"A connection error occurred: {e}")
    except exceptions.NoDataError as e:
        print(f"No data found: {e}")
    except ValueError as e: 
        print(f"A value error occurred: {e}. Check RSSD ID, reporting period, or credentials.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("Skipping `collect_data` examples as prerequisites are not met.")

Fetching Call data for RSSD 37 for period 12/31/2024 as Pandas DataFrame...
Successfully fetched Call data. Shape: (732, 8)
First 5 rows:
       mdrm rssd    quarter  int_data  float_data bool_data str_data data_type
0  RIADFT29   37 2024-12-31       0.0         NaN      None     None       int
1  RIADJH89   37 2024-12-31       0.0         NaN      None     None       int
2  RCONF047   37 2024-12-31       0.0         NaN      None     None       int
3  RCON8783   37 2024-12-31     797.0         NaN      None     None       int
4  RCON8782   37 2024-12-31     984.0         NaN      None     None       int

Fetching UBPR data for RSSD 37 for period 12/31/2024 as list...
Successfully fetched UBPR data. Number of items: 2930
First item:
{'mdrm': 'RCOAJJ29', 'rssd': '37', 'quarter': datetime.datetime(2024, 12, 31, 0, 0), 'int_data': None, 'float_data': 0.0, 'bool_data': None, 'str_data': None, 'data_type': 'float'}

Fetching Call data for RSSD 37 for period 12/31/2024 as Polars DataFrame...

## 4. Collect Filers Since Date

The `collect_filers_since_date` function retrieves the RSSD IDs of reporters who have filed Call Reports *after* a given `since_date` for a specific `reporting_period`.

In [21]:
if 'creds' in locals() and 'ffiec_session' in locals() and 'call_periods_list' in locals() and call_periods_list:
    # Reporting period for which to check filings
    REPORTING_PERIOD_FOR_FILERS = call_periods_list[0] # e.g., "03/31/2023" or "1Q2023"
    
    # Date after which filings should be checked. Can be YYYY-MM-DD, YYYYMMDD, MM/DD/YYYY, or datetime object.
    SINCE_DATE_EXAMPLE = "04/01/2023" # Example: check for filings after April 1st for Q1 report
    # Or use a datetime object
    # from datetime import datetime
    # SINCE_DATE_EXAMPLE = datetime(2023, 4, 1) 
    
    try:
        print(f"Fetching filers for period {REPORTING_PERIOD_FOR_FILERS} since {SINCE_DATE_EXAMPLE} (as Polars Series)...")
        filers_polars = methods.collect_filers_since_date(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_FOR_FILERS,
            since_date=SINCE_DATE_EXAMPLE,
            output_type="polars"
        )
        if filers_polars is not None and len(filers_polars) > 0:
            print(f"Found {len(filers_polars)} filers. First 5:\n{filers_polars.head()}")
        else:
            print("No filers found for the specified criteria.")
            
    except requests.exceptions.ConnectionError as e:
        print(f"A connection error occurred: {e}")
    except exceptions.NoDataError as e:
        print(f"No data found: {e}")
    except ValueError as e: 
        print(f"A value error occurred: {e}. Check date formats or credentials.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("Skipping `collect_filers_since_date` examples as prerequisites are not met.")

Fetching filers for period 3/31/2025 since 04/01/2023 (as Polars Series)...
Found 4519 filers. First 5:
shape: (10,)
Series: 'rssd_id' [i64]
[
	688556
	175458
	126049
	469250
	723158
	1014246
	92144
	952257
	856159
	741358
]


## 5. Collect Filers Submission Date and Time

The `collect_filers_submission_date_time` function retrieves the RSSD IDs and the exact submission date and time (in US/Eastern timezone) for reporters who filed Call Reports *after* a given `since_date` for a specific `reporting_period`.

In [24]:
if 'creds' in locals() and 'ffiec_session' in locals() and 'call_periods_list' in locals() and call_periods_list:
    REPORTING_PERIOD_SUBMISSION = call_periods_list[0] # e.g., "03/31/2023"
    SINCE_DATE_SUBMISSION = "04/01/2023" # e.g., check submissions after April 1st, 2023
    
    try:
        print(f"Fetching filer submission date/times for {REPORTING_PERIOD_SUBMISSION} since {SINCE_DATE_SUBMISSION} (as Pandas DataFrame, Python datetime)...")
        filer_submissions_df = methods.collect_filers_submission_date_time(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_SUBMISSION,
            since_date=SINCE_DATE_SUBMISSION,
            output_type="pandas",
            date_output_format="python_format" # Get datetimes as timezone-aware Python datetime objects
        )
        if not filer_submissions_df.empty:
            print(f"Found {len(filer_submissions_df)} submissions. First 5:\n{filer_submissions_df.head()}")
            # You can see the timezone info:
            if 'datetime' in filer_submissions_df.columns and not filer_submissions_df['datetime'].empty:
                 print(f"Timezone of first datetime: {filer_submissions_df['datetime'].iloc[0].tzinfo}")
        else:
            print("No filer submissions found for the specified criteria.")
            
        # Example 2: Filers submission date/times as a Polars Series
        print(f"\nFetching filer submission date/times for {REPORTING_PERIOD_SUBMISSION} since {SINCE_DATE_SUBMISSION} (as Polars Series)...")
        filers_polars = methods.collect_filers_submission_date_time(
            session=ffiec_session,
            creds=creds,
            reporting_period=REPORTING_PERIOD_SUBMISSION,
            since_date=SINCE_DATE_SUBMISSION,
            output_type="polars",
            date_output_format="python_format"
        )
        if filers_polars is not None and len(filers_polars) > 0:
            print(f"Found {len(filers_polars)} filers. First 5:\n{filers_polars.head()}")
        else:
            print("No filers found for the specified criteria.")
            
    except requests.exceptions.ConnectionError as e:
        print(f"A connection error occurred: {e}")
    except exceptions.NoDataError as e:
        print(f"No data found: {e}")
    except ValueError as e: 
        print(f"A value error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("Skipping `collect_filers_submission_date_time` examples as prerequisites are not met.")

Fetching filer submission date/times for 3/31/2025 since 04/01/2023 (as Pandas DataFrame, Python datetime)...
Found 4519 submissions. First 5:
     rssd                  datetime
0  688556 2025-04-01 12:03:25-04:00
1  175458 2025-04-01 07:55:30-04:00
2  126049 2025-04-01 03:59:08-04:00
3  469250 2025-04-01 04:28:40-04:00
4  723158 2025-04-01 04:59:55-04:00
Timezone of first datetime: US/Eastern

Fetching filer submission date/times for 3/31/2025 since 04/01/2023 (as Polars Series)...
Found 4519 filers. First 5:
shape: (5, 2)
┌─────────┬──────────────────────────┐
│ rssd_id ┆ datetime                 │
│ ---     ┆ ---                      │
│ null    ┆ datetime[μs, US/Eastern] │
╞═════════╪══════════════════════════╡
│ null    ┆ 2025-04-01 12:03:25 EDT  │
│ null    ┆ 2025-04-01 07:55:30 EDT  │
│ null    ┆ 2025-04-01 03:59:08 EDT  │
│ null    ┆ 2025-04-01 04:28:40 EDT  │
│ null    ┆ 2025-04-01 04:59:55 EDT  │
└─────────┴──────────────────────────┘


## 6. Collect Filers on Reporting Period (Panel of Reporters)

The `collect_filers_on_reporting_period` function retrieves a list of all financial institutions in the Panel of Reporters for a given `reporting_period`. This includes details about the institution and whether they have filed for that period. This is for Call Reports only.

In [26]:
if 'creds' in locals() and 'ffiec_session' in locals() and 'call_periods_list' in locals() and call_periods_list:
    PANEL_REPORTING_PERIOD = call_periods_list[0] # e.g., "03/31/2023"
    
    try:
        print(f"Fetching panel of reporters for {PANEL_REPORTING_PERIOD} (as list of dicts)...")
        panel_reporters_list = methods.collect_filers_on_reporting_period(
            session=ffiec_session,
            creds=creds,
            reporting_period=PANEL_REPORTING_PERIOD,
            output_type="list"
        )
        if panel_reporters_list:
            print(f"Found {len(panel_reporters_list)} reporters in the panel. First one:\n{panel_reporters_list[0]}")
        else:
            print("No panel of reporters found (this is unlikely for a valid period).")

        # Example as Pandas DataFrame
        print(f"\nFetching panel of reporters for {PANEL_REPORTING_PERIOD} (as Pandas DataFrame)...")
        panel_reporters_df = methods.collect_filers_on_reporting_period(
            session=ffiec_session,
            creds=creds,
            reporting_period=PANEL_REPORTING_PERIOD,
            output_type="pandas"
        )
        if not panel_reporters_df.empty:
            print(f"Found {len(panel_reporters_df)} reporters. First 5:\n{panel_reporters_df.head()}")
        else:
            print("No panel of reporters found as DataFrame.")
            
        # Example as Polars Series
        print(f"\nFetching panel of reporters for {PANEL_REPORTING_PERIOD} (as Polars Series)...")
        panel_reporters_polars = methods.collect_filers_on_reporting_period(
            session=ffiec_session,
            creds=creds,
            reporting_period=PANEL_REPORTING_PERIOD,
            output_type="polars"
        )
        if panel_reporters_polars is not None and len(panel_reporters_polars) > 0:
            print(f"Found {len(panel_reporters_polars)} reporters. First 5:\n{panel_reporters_polars.head()}")
        else:
            print("No panel of reporters found as Polars Series.")
            
            
    except requests.exceptions.ConnectionError as e:
        print(f"A connection error occurred: {e}")
    except exceptions.NoDataError as e: # Should not happen for this endpoint if period is valid
        print(f"No data found: {e}")
    except ValueError as e: 
        print(f"A value error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print("Skipping `collect_filers_on_reporting_period` examples as prerequisites are not met.")

Fetching panel of reporters for 3/31/2025 (as list of dicts)...
Found 4519 reporters in the panel. First one:
{'id_rssd': '37', 'fdic_cert_number': '10057', 'occ_chart_number': None, 'ots_dock_number': '16553', 'primary_aba_rout_number': '61107146', 'name': 'BANK OF HANCOCK COUNTY', 'state': 'GA', 'city': 'SPARTA', 'address': '12855 BROAD STREET', 'filing_type': '051', 'has_filed_for_reporting_period': True}

Fetching panel of reporters for 3/31/2025 (as Pandas DataFrame)...
Found 4519 reporters. First 5:
  id_rssd fdic_cert_number occ_chart_number ots_dock_number  \
0      37            10057             None           16553   
1     242             3850             None            None   
2     279            28868             None            2523   
3     354            14083             None            None   
4     457            10202             None            None   

  primary_aba_rout_number                              name state     city  \
0                61107146       

## 7. Error Handling

The library uses standard Python exceptions (e.g., `ValueError` for bad inputs, `requests.exceptions.ConnectionError` for network issues) and may also raise exceptions from the underlying `zeep` SOAP library if the webservice returns an error.

The `ffiec_data_connect.exceptions` module defines custom exceptions like `NoDataError`, although in practice, the FFIEC service might return empty lists/dataframes or raise `zeep` errors for 'no data' scenarios depending on the endpoint.

In [27]:
try:
    # Example of an operation that might fail if parameters are incorrect or service is down
    # methods.collect_data(ffiec_session, creds, "invalid_period_format", "0", "call")
    print("Simulated error handling section complete.")
    # For a real test, uncomment and use an invalid call. For now, we just show the structure.
    if 'creds' in locals() and 'ffiec_session' in locals():
        # Example trying to fetch data for a clearly invalid RSSD and period
        methods.collect_data(ffiec_session, creds, "1Q1900", "0", "call")
except ValueError as ve:
    print(f"Caught a ValueError: {ve}")
except exceptions.NoDataError as nde:
    print(f"Caught a NoDataError: {nde}")
except requests.exceptions.RequestException as re:
    print(f"Caught a RequestException (network/connection issue): {re}")
except Exception as e:
    # This will catch other errors, including those from the Zeep library (SOAP faults)
    print(f"Caught a general FFIEC API or other error: {e}")

Simulated error handling section complete.
Caught a general FFIEC API or other error: Invalid Reporting Period End Date specified in request.


## 8. Helper Functions & Further Details

The `ffiec_data_connect` library contains various internal helper functions (e.g., for date conversions in `methods.py` and `xbrl_processor.py`, or credential validation in `credentials.py`). Generally, you won't need to call these directly as they are used by the main public-facing functions.

For more details on specific parameters, valid date formats, and output structures, refer to the docstrings of each function within the `ffiec_data_connect.methods` module.